In [1]:
import keras
import string
import numpy as np
import random
import sys
import io

Using TensorFlow backend.


In [2]:
def load_word_list(path):
    """
    Loads a list of the words from the file at path <path>, removing all
    non-alpha-numeric characters from the file.
    """
    with open(path) as handle:
        # Load a list of whitespace-delimited words from the specified file
        raw_text = handle.read().strip().split()
        # Strip non-alphanumeric characters from each word
        alphanumeric_words = map(lambda word: ''.join(char for char in word if char.isalnum()), raw_text)
        # Filter out words that are now empty (e.g. strings that only contained non-alphanumeric chars)
        alphanumeric_words = filter(lambda word: len(word) > 0, alphanumeric_words)
        # Convert each word to lowercase and return the result
        return list(map(lambda word: word.lower(), alphanumeric_words))
    
def list_string(word_list):
    '''
    Converts a list of words into a string.
    '''
    word_string = ' '
    return word_string.join(word_list)

In [3]:
#loading the text
word_list = load_word_list('./data/shakespeare.txt')

#removing sonnet numbers
for i in range(1,155):
    word_list.remove('{}'.format(i))

#converting the stripped wordlist back to a string
word_string = list_string(word_list)

In [4]:
#getting all characters in the text and assigning indices
characters = sorted(list(set(word_string)))
character_indices = dict((c, i) for i, c in enumerate(characters))
indices = dict((i, c) for i, c in enumerate(characters))

In [5]:
#splitting the text into sequences of fixed length
max_length = 40
#pick sequences starting at each 5th character
step_size = 5
sentences = []
next_characters = []
for i in range(0, len(word_string) - max_length, step_size):
    sentences.append(word_string[i: i + max_length])
    next_characters.append(word_string[i + max_length])

In [6]:
x = np.zeros((len(sentences), max_length, len(characters)), dtype=np.bool)
y = np.zeros((len(sentences), len(characters)), dtype=np.bool)

for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, character_indices[char]] = 1
    y[i, character_indices[next_characters[i]]] = 1

In [7]:
model = keras.models.Sequential()
model.add(keras.layers.LSTM(180, input_shape=(max_length, len(characters))))
model.add(keras.layers.Dense(len(characters), activation='softmax'))

optimizer = keras.optimizers.adam(learning_rate=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 180)               149760    
_________________________________________________________________
dense_1 (Dense)              (None, 27)                4887      
Total params: 154,647
Trainable params: 154,647
Non-trainable params: 0
_________________________________________________________________
None


In [8]:
def sample_with_temp(predictions, temperature=1.0):
    predictions = np.asarray(predictions).astype('float64')
    predictions = np.log(predictions) / temperature
    exp_predictions = np.exp(predictions)
    predictions = exp_predictions / np.sum(exp_predictions)
    probas = np.random.multinomial(1, predictions, 1)
    return np.argmax(probas)

### Version with random sentence seed

In [9]:
def model_compile_random(epoch, _):
    # after each epoch print the generated sonnet text
    print()
    print('Generating sonnet after Epoch: %d' % epoch)

    start_index = random.randint(0, len(word_string) - max_length - 1)
    for temps in [1.5, 0.75, 0.25]:
        print('Temperature:', temps)

        new_sentence = ''
        sentence = word_string[start_index: start_index + max_length]
        new_sentence += sentence
        print('Generating sonnet with seed clause: "' + sentence + '"')
        sys.stdout.write(new_sentence)

        for i in range(400):
            x_pred = np.zeros((1, max_length, len(characters)))
            for t, char in enumerate(sentence):
                x_pred[0, t, character_indices[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample_with_temp(preds, temps)
            next_char = indices[next_index]

            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()


In [10]:
print_callback = keras.callbacks.LambdaCallback(on_epoch_end=model_compile_random)

model.fit(x, y, epochs=50, callbacks=[print_callback])

Epoch 1/50
18100/18100 [==============================] - 15s 818us/step - loss: 2.2530

Generating sonnet after Epoch: 0
Temperature: 1.5
Generating sonnet with seed clause: "ace hath not the power to make love groa"
ace hath not the power to make love groadiyesy basprore tithn ge in nkruge qrtwly diwktysweby tnim se wakillsj evearch jetdel and wayty onw oqwlh yomy loverfy datace kairy onp yetf wich dookd to mrisjt in willpsnxar chet llds pnict mige mowre osu sadt of qutuncud nrmarn adcy lawspunt thourasle khist grdo ifffmy gozty foinc maghs whom ffrith for  theigod hleanspe naazge tsfecy eludesly imutas  watt bed exofp lowae qefe fo  thatbabnciniwi
Temperature: 0.75
Generating sonnet with seed clause: "ace hath not the power to make love groa"
ace hath not the power to make love groaghs levawmer com the and iny once in to moth not haxp hasf nour dethe to in to and thy in womy thee wind ag the live worl all y and dreast my ve and prome vat that suth and and my and rowh seece and thy a

/Users/nicholassarai/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: RuntimeWarning: divide by zero encountered in log
  This is separate from the ipykernel package so we can avoid doing imports until


age can drown ele sacce but beforn who longe assee and right ml grow at prink neclse where that you aly
Temperature: 0.75
Generating sonnet with seed clause: "ered hours but all alone stands hugely p"
ered hours but all alone stands hugely pain to ment frown the clours and ower be confare to may ble as pleass the liven sprear a perw or ey to times in thing in sple will the tomour clous or ey of my veire and do not the livent and sight cannor shath for thy my veares in shadow did erre dread and dosn thy most break which husband and sin wond that stould to upon my sist as fair the dayow of mine eyes meny are than light o truns in thine
Temperature: 0.25
Generating sonnet with seed clause: "ered hours but all alone stands hugely p"
ered hours but all alone stands hugely pain that is busi is canst thou worths in thy mears to seef a dect the tomene in thine eam hath the this thou being face faire and sime the sun disdane or you aloor to me it and sime you too ill ryight and therefore my sel

### Version with static seed

In [11]:
def model_compile_static(epoch, _):
    # after each epoch print the generated sonnet text
    print()
    print('Generating sonnet after Epoch: %d' % epoch)

    start_index = random.randint(0, len(word_string) - max_length - 1)
    for temps in [1.5, 0.75, 0.25]:
        print('Temperature:', temps)

        new_sentence = ''
        sentence = "shall i  compare  thee  to a summers day"
        new_sentence += sentence
        print('Generating sonnet with seed clause: "' + sentence + '"')
        sys.stdout.write(new_sentence)

        for i in range(400):
            x_pred = np.zeros((1, max_length, len(characters)))
            for t, char in enumerate(sentence):
                x_pred[0, t, character_indices[char]] = 1.

            predictions = model.predict(x_pred, verbose=0)[0]
            next_index = sample_with_temp(predictions, temps)
            next_char = indices[next_index]

            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()


In [12]:
print_callback = keras.callbacks.LambdaCallback(on_epoch_end=model_compile_static)

model.fit(x, y, epochs=50, callbacks=[print_callback])

Epoch 1/50
18100/18100 [==============================] - 14s 792us/step - loss: 0.7365

Generating sonnet after Epoch: 0
Temperature: 1.5
Generating sonnet with seed clause: "shall i  compare  thee  to a summers day"
shall i  compare  thee  to a summers day make thereins he blt upontrand no oto made long agains sprefous than if o you a pet your close thou hamow shol is from moste is graflore founded nor you greald wason hath my on by heart for ihade best my must is my joouchs in ereming fou thy so lond mone date rements is thee ablon s a vance old quitiegh in enchefest thine eyes deewoul wild my bodd andered other hath much neesuil ornan cropme my m
Temperature: 0.75
Generating sonnet with seed clause: "shall i  compare  thee  to a summers day"
shall i  compare  thee  to a summers day dast she from youths with thine tome the moun which che thou harth as prefent me ist my more spirit once there with fure thou are to ance the thou a twinks to trees whtlly hes i love wherein that disare 

/Users/nicholassarai/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: RuntimeWarning: divide by zero encountered in log
  This is separate from the ipykernel package so we can avoid doing imports until


till tell my vowe them in their hour set he bave rendem with thine eye prissing thee and made longeler and gain what puting thee as fair and their rost i tol the rest where the sentlest love where more decgasirs firs of their love the cannot thou harty more and there in mine eye them from hand in me
Temperature: 0.25
Generating sonnet with seed clause: "shall i  compare  thee  to a summers day"
shall i  compare  thee  to a summers days in eyes with the better so wond or eye les and in the live thee and so where my from where what you hove your self so love be poor in such strength the selss thou a thou alt rest leaves and their part and therefore them i am so sweet love where me bours be chose thee as thy less thou be const the sell my so love be poor he when i sight be chorly i to my live thee all the wired so rand so for thy
Epoch 3/50
18100/18100 [==============================] - 14s 790us/step - loss: 0.7591

Generating sonnet after Epoch: 2
Temperature: 1.5
Generating sonnet with